# 360_Bangkok
This note book uses to download 360 images taken by GoPro Max camera mounted on a helmet while riding a motorcycle from December 18, 2022, to January 16, 2023.

## How to download Street View Images (SVIs) from Mapillary

https://www.mapillary.com/developer/api-documentation

### You need to create an account and get an API key from here: 
https://www.mapillary.com/dashboard/developers


In [2]:
# List of access tokens
tokens = ["List of access tokens here"]


## Download images from Mapillary

### creator_id = 104320792344944

### Download geojson (information)

In [8]:
import os
import requests
import mercantile
import json
from vt2geojson.tools import vt_bytes_to_geojson
import datetime
import random

# Set up paths and folders
folder_name = './/360_Bangkok'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Select a random token
selected_token = random.choice(tokens)

# Headers for requests
headers = {'Authorization': f'OAuth {selected_token}'}

# Initialize an empty GeoJSON for output
output = {"type": "FeatureCollection", "features": []}

# Date range filter: December 18, 2022, to January 16, 2023
start_date = datetime.datetime(2022, 12, 18)
end_date = datetime.datetime(2023, 1, 16)

# Filter creator_id
filter_creator_id = 104320792344944

# Define the bounding box (west, south, east, north) for the area of interest
bounding_box = (100.450229, 13.695348, 100.633392, 13.918410)

# Collect tiles at zoom level 14 within the bounding box
tiles = list(mercantile.tiles(*bounding_box, 14))

# Loop through the tiles and fetch data
for tile in tiles:
    tile_url = f'https://tiles.mapillary.com/maps/vtp/mly1_public/2/{tile.z}/{tile.x}/{tile.y}?access_token={selected_token}'
    response = requests.get(tile_url)
    if response.status_code == 200:
        data = vt_bytes_to_geojson(response.content, tile.x, tile.y, tile.z, layer="image")

        # Process each feature in the geojson
        for feature in data['features']:
            coords = feature['geometry']['coordinates']
            properties = feature['properties']
            lng, lat = coords
            sequence_id = properties['sequence_id']
            image_id = properties['id']
            captured_at = properties['captured_at']
            compass_angle = properties['compass_angle']
            is_pano = properties['is_pano']
            creator_id = properties.get('creator_id')

            capture_date = datetime.datetime.fromtimestamp(int(captured_at) / 1000)  # assuming captured_at is in milliseconds

            # Ensure the feature matches the creator_id, and falls within the date range
            if creator_id == filter_creator_id and start_date <= capture_date <= end_date:
                # Append metadata to output geojson
                feature['properties'].update({
                    "captured_at": captured_at,
                    "compass_angle": compass_angle,
                    "is_pano": is_pano,
                    "image_id": image_id  # Add image ID to the properties (instead of the image URL)
                })
                output['features'].append(feature)

# Save the GeoJSON object to a file
geojson_path = os.path.join(folder_name, 'mapillary_metadata.geojson')
with open(geojson_path, 'w') as file:
    json.dump(output, file, indent=4)  # Use indent=4 for better readability

print(f"GeoJSON data saved to {geojson_path}")


GeoJSON data saved to .//360_Bangkok\mapillary_metadata.geojson


### Download images

In [ ]:
import os
import requests
import mercantile
import json
from vt2geojson.tools import vt_bytes_to_geojson
import datetime
import random

# Set up paths and folders
folder_name = './/360_Bangkok'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Select a random token
selected_token = random.choice(tokens)

# Headers for image download requests
headers = {'Authorization': f'OAuth {selected_token}'}

# Initialize an empty GeoJSON for output
output = {"type": "FeatureCollection", "features": []}

# Date range filter: December 18, 2022, to January 16, 2023
start_date = datetime.datetime(2022, 12, 18)
end_date = datetime.datetime(2023, 1, 16)

# Filter creator_id
filter_creator_id = 104320792344944
#13.918410, 100.450229
#13.695348, 100.633392
# Define the bounding box (west, south, east, north) for the area of interest
bounding_box = (100.450229, 13.695348, 100.633392,13.918410 )  # Replace with actual coordinates

# Collect tiles at zoom level 14 within the bounding box
tiles = list(mercantile.tiles(*bounding_box, 14))

# Loop through the tiles and fetch data
for tile in tiles:
    tile_url = f'https://tiles.mapillary.com/maps/vtp/mly1_public/2/{tile.z}/{tile.x}/{tile.y}?access_token={selected_token}'
    response = requests.get(tile_url)
    if response.status_code == 200:
        data = vt_bytes_to_geojson(response.content, tile.x, tile.y, tile.z, layer="image")

        # Process each feature in the geojson
        for feature in data['features']:
            coords = feature['geometry']['coordinates']
            properties = feature['properties']
            lng, lat = coords
            sequence_id = properties['sequence_id']
            image_id = properties['id']
            captured_at = properties['captured_at']
            compass_angle = properties['compass_angle']
            is_pano = properties['is_pano']
            creator_id = properties.get('creator_id')

            capture_date = datetime.datetime.fromtimestamp(int(captured_at) / 1000)  # assuming captured_at is in milliseconds

            # Ensure the feature matches the creator_id, and falls within the date range
            if creator_id == filter_creator_id and start_date <= capture_date <= end_date:
                # Fetch the image URL
                image_info_url = f'https://graph.mapillary.com/{image_id}?fields=id,thumb_2048_url&access_token={selected_token}'
                image_info_response = requests.get(image_info_url, headers=headers)
                if image_info_response.status_code == 200:
                    image_info = image_info_response.json()
                    image_url = image_info['thumb_2048_url']

                    # Download the image and save it with metadata
                    image_response = requests.get(image_url, stream=True)
                    if image_response.status_code == 200:
                        image_path = os.path.join(folder_name, f'{image_id}.jpg')
                        with open(image_path, 'wb') as file:
                            file.write(image_response.content)

                        # Append metadata to output geojson
                        feature['properties'].update({
                            "captured_at": captured_at,
                            "compass_angle": compass_angle,
                            "is_pano": is_pano,
                            "image_url": image_url  # Add image URL to the properties
                        })
                        output['features'].append(feature)

# Save the GeoJSON object to a file
geojson_path = os.path.join(folder_name, 'mapillary_metadata.geojson')
with open(geojson_path, 'w') as file:
    json.dump(output, file, indent=4)  # Use indent=4 for better readability

print(f"GeoJSON data saved to {geojson_path}")
